In [1]:
import requests
import csv
import pandas as pd
import re

In [2]:
url='http://compras.dados.gov.br/licitacoes/v1/licitacoes.csv'
data_min = '20180101'
data_max = '20181231'
resultado = pd.DataFrame()

In [3]:
def convertRequestToCsv(response):
    decoded_content = response.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    df = pd.DataFrame(my_list)
    ##Sei que não é o certo, mas é o que pude resolver no momento
    colunas= df.iloc[0]
    novo  = pd.DataFrame(df.values[1:], columns=colunas)
    return novo

In [4]:
def listarLicitacoes(valor, url):
    frames = []
    for x in range(0, valor, 500):
        payload = {'data_publicacao_min': data_min,'data_publicacao_max': data_max, 'offset': x}
        response = requests.get(url, params=payload)
        data = convertRequestToCsv(response)
        frames.append(data)
    result = pd.concat(frames)
    result = result[['UASG', 'Modalidade da Licitação', 'Identificador da Licitação', 
                     'Número de Itens na Licitação'] ]
    return result

In [5]:
def listaItensLicitacao(valor, licitacao):
    link = 'http://compras.dados.gov.br/licitacoes/doc/licitacao/'+licitacao+'/itens.csv'
    lista = pd.read_csv(link)
    lista = lista[['Número da Licitação', 'Número do Item na Licitação', 'Descrição do Item','CNPJ do Vencedor']]
    lista['Número da Licitação'] = licitacao
    lista.rename(columns={'Número da Licitação': 'Identificador da Licitação'}, inplace = True)
    
    for index, row in lista.iterrows():
        if(str(row['CNPJ do Vencedor']) != 'nan'):
            cnpj = str(row['CNPJ do Vencedor'])
            cnpj, descartar = cnpj.split(':')
            descartar, cnpj = cnpj.split(' ')
            cnpj = re.sub('[!./-]', '', cnpj)
            lista.loc[ index, 'CNPJ do Vencedor' ] = str( cnpj)
            
    return lista

In [6]:
def concatenacaoListasItensLicitacao(data):
    listaItens = []
    for index, row in data.iterrows():
        lista = listaItensLicitacao(row['Número de Itens na Licitação'], str(row['Identificador da Licitação']))
        listaItens.append(lista)
        print(index)
    result = pd.concat(listaItens)
    return result

In [7]:
def getFornecedor(cnpj):
    cnpj = re.sub('[!./-]', '', cnpj)
    link = 'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'+cnpj+'.csv'
    fornecedor = pd.read_csv(link)
    fornecedor['CNPJ do Vencedor'] = cnpj
    fornecedor = fornecedor[['CNPJ do Vencedor','CNPJ', 'Razão Social']]
    fornecedor.rename(columns={'CNPJ': 'CNPJ do Fornecedor', 'Razão Social' :'Razão Social do Fornecedor'}, 
                      inplace = True)
    return fornecedor

In [8]:
def listaCompleta(data,resultado):
    juncao = pd.merge(data,resultado,on='Identificador da Licitação')
    return juncao

In [9]:
def listaCompletaComCNPJ(data,resultado):
    juncao = pd.merge(data,resultado,on='Identificador da Licitação')
    juncao.dropna(axis=0, how='any',inplace = True)
    return juncao

In [10]:
data = listarLicitacoes(valor=500, url=url)

In [11]:
data = data[2:110]
data.head(15)

,UASG,Modalidade da Licitação,Identificador da Licitação,Número de Itens na Licitação
2,160272: 21 GRUPO DE ARTILHARIA CAMPANHA/RJ,5: PREGÃO ...,16027205000012017,30
3,154042: UNIVERSIDADE FEDERAL DO RIO GRANDE,99: RDC,15404299000012017,1
4,765700: CENTRO MEDICO ASSISTENCIAL DA MARINHA,2: TOMADA DE PREÇOS ...,76570002000012017,3
5,788820: CENTRO DE INTENDENCIA DA MARINHA EM MA...,5: PREGÃO ...,78882005000012017,135
6,160039: HOSPITAL GERAL DE SALVADOR,2: TOMADA DE PREÇOS ...,16003902000012017,1
7,160203: 2 BATALHAO DE ENGENHARIA DE CONSTRUCAO,3: CONCORRÊNCIA ...,16020303000012017,1
8,"158431: INST.FED.DE EDUC., CIENC.E TEC DE GO/C...",1: CONVITE ...,15843101000012017,1
9,158469: INST FED.DA PARAIBA/CAMPUS JOÃO PESSOA,5: PREGÃO ...,15846905000012017,1
10,257049: DISTRITO SANIT.ESP.INDÍGENA - PORTO VELHO,1: CONVITE ...,25704901000012017,1
11,160318: ESCOLA DE SARGENTOS DE LOGISTICA,5: PREGÃO ...,16031805000012017,28


In [19]:
resultado.head(10)

,Identificador da Licitação,Número do Item na Licitação,Descrição do Item,CNPJ do Vencedor
0,16027205000012017,1,"AGENDA, Agenda personalizada Ano 2017 ou 2018 ...",NaN
1,16027205000012017,2,NaN,NaN
2,16027205000012017,3,NaN,NaN
3,16027205000012017,4,Caixa porta vinho em madeira ou MDF revestido ...,NaN
4,16027205000012017,5,NaN,NaN
5,16027205000012017,6,MEMÓRIA FLASH: Conjunto com pendrive com capac...,NaN
6,16027205000012017,7,"Porta bolsa, tipo gancho em metal resistente, ...",NaN
7,16027205000012017,8,"CAMISA UNIFORME, CAMISA MASCULINA: Camisa tipo...",NaN
8,16027205000012017,9,"CAMISA UNIFORME, Camiseta em tecido 100% polié...",NaN
9,16027205000012017,10,"CHAVEIRO, Chaveiro em metal medindo 45 mm x 30...",NaN


In [14]:
juncao = listaCompleta(data,resultado)

In [22]:
juncao.head()

,UASG,Modalidade da Licitação,Identificador da Licitação,Número de Itens na Licitação,Número do Item na Licitação,Descrição do Item,CNPJ do Vencedor
30,154042: UNIVERSIDADE FEDERAL DO RIO GRANDE,99: RDC,15404299000012017,1,1,Contratação integrada de empresa especializada...,03769527000173
169,160039: HOSPITAL GERAL DE SALVADOR,2: TOMADA DE PREÇOS ...,16003902000012017,1,1,Contratação de empresa especializada na execuç...,07107483000177
170,160203: 2 BATALHAO DE ENGENHARIA DE CONSTRUCAO,3: CONCORRÊNCIA ...,16020303000012017,1,1,Construção do Pavilhão de Administrativo do 25...,07607937000179
171,"158431: INST.FED.DE EDUC., CIENC.E TEC DE GO/C...",1: CONVITE ...,15843101000012017,1,1,1.1. Adequar e concluir toda a rede elétrica d...,28331609000162
172,158469: INST FED.DA PARAIBA/CAMPUS JOÃO PESSOA,5: PREGÃO ...,15846905000012017,1,1,Execução da Semana Esportiva e Cultural do IFP...,05927075000136


In [16]:
juncao = listaCompletaComCNPJ(data,resultado)

In [20]:
juncao.head(10)

,UASG,Modalidade da Licitação,Identificador da Licitação,Número de Itens na Licitação,Número do Item na Licitação,Descrição do Item,CNPJ do Vencedor
30,154042: UNIVERSIDADE FEDERAL DO RIO GRANDE,99: RDC,15404299000012017,1,1,Contratação integrada de empresa especializada...,03769527000173
169,160039: HOSPITAL GERAL DE SALVADOR,2: TOMADA DE PREÇOS ...,16003902000012017,1,1,Contratação de empresa especializada na execuç...,07107483000177
170,160203: 2 BATALHAO DE ENGENHARIA DE CONSTRUCAO,3: CONCORRÊNCIA ...,16020303000012017,1,1,Construção do Pavilhão de Administrativo do 25...,07607937000179
171,"158431: INST.FED.DE EDUC., CIENC.E TEC DE GO/C...",1: CONVITE ...,15843101000012017,1,1,1.1. Adequar e concluir toda a rede elétrica d...,28331609000162
172,158469: INST FED.DA PARAIBA/CAMPUS JOÃO PESSOA,5: PREGÃO ...,15846905000012017,1,1,Execução da Semana Esportiva e Cultural do IFP...,05927075000136
202,160501: MUSEU HISTORICO DO EX. FORTE COPACABAN...,2: TOMADA DE PREÇOS ...,16050102000012017,1,1,Obra de restauração da rede de esgoto do Museu...,06241337000177
203,150229: HOSPITAL UNIVERSITARIO PROF. ALBERTO A...,99: RDC,15022999000012017,1,1,1.1 Contratação Integrada de empresa para pres...,03486715000194
204,150246: MATERNIDADE ESCOLA ASSIS CHATEAUBRIAND,99: RDC,15024699000012017,1,1,O objeto deste Projeto Básico é a contratação ...,19556837000140
241,160334: POLICLINICA MILITAR DO RIO DE JANEIRO/RJ,2: TOMADA DE PREÇOS ...,16033402000012017,1,1,SERVIÇO DE ADEQUAÇÃO AS INSTALAÇÕES ELÉTRICAS ...,22064269000174
285,511180: GERÊNCIA EXECUTIVA RECIFE/PE-INSS,3: CONCORRÊNCIA ...,51118003000012017,1,1,Contratação de empresa de engenharia para exec...,25018526000120
